In [2]:
from gensim.models.keyedvectors import KeyedVectors
from tqdm import tqdm
import numpy as  np
import os
import re
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
import pandas as pd

path_model = r"./word2vec_twitter_model.bin"
path_data=r"./data"

[nltk_data] Downloading package punkt to /home/ors/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/ors/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
def clean_text(text):
    text = text.replace("\\", " ").replace(u"╚", " ").replace(u"╩", " ")
    text = text.lower()
    text = re.sub('\-\s\r\n\s{1,}|\-\s\r\n|\r\n', '', text) 
    text = re.sub('[.,:;_%©?*,!@#$%^&()\d]|[+=]|[[]|[]]|[/]|"|\s{2,}|-', ' ', text)
    words = text.split()
    words = [w for w in words if len(w)>3]
    stop_words = set(stopwords.words('english'))
    words = [w for w in words if not w in stop_words]
    text=' '.join(words)
    tokens = word_tokenize(text)
    porter = PorterStemmer()
    stemmed = [porter.stem(word) for word in tokens]
    return stemmed
    
words=set()#множество слов по всем текстам

for f in os.listdir(path_data):
    print(f)
    full_path = path_data+'/'+f
    try:
        f_text = open(full_path, "r").readlines()
        for line in f_text:
            words_text=clean_text(line)
            words.update(words_text)
    except Exception as  err:
        print(err)
        
        pass
        '''
        lines=[]
        with open(full_path) as file:
            while True:
                try:
                    line = file.readline()
                    lines.append(line)
                except:
                    pass
        for line in lines:
            words_text=clean_text(line)
            words.update(words_text)
        '''
    
print("Всего слов: {}".format(len(words)))

nm_train.txt
np_val.txt
p_test.txt
i_test.txt
ni_train.txt
np_train.txt
nm_val.txt
ni_test.txt
i_val.txt
np_test.txt
nm_test.txt
none-class-1200.txt
p_val.txt
m_train.txt
i_train.txt
csv
[Errno 21] Is a directory: './data/csv'
m_val.txt
m_test.txt
p_train.txt
ni_val.txt
Всего слов: 10666


In [3]:
embdict=dict()#словарь эмбеддингов и слов
index=0
porter = PorterStemmer()

with open(path_model,'rb')as f:
    header = f.readline()
    vocab_size, layer1_size = map(int, header.split())
    binary_len = np.dtype('float32').itemsize * layer1_size
    for line in range(vocab_size):
        word = []
        while True:
            ch = f.read(1).decode(errors='ignore')
            if ch ==' ':
                word = ''.join(word)
                break
            if ch != '\n':
                word.append(ch)
        if len(word) != 0:
            tp= np.fromstring(f.read(binary_len), dtype='float32')
            word = porter.stem(word.lower())
            if word in words:
                embdict[str(word)]=tp.tolist()

        else:
            f.read(binary_len)
        index+=1
        if index%1000000==0:
            print("iteration "+str(index))
print(embdict)
print("Слов в словаре:"+str(len(embdict)))

/home/ors/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead


iteration 1000000
iteration 2000000
iteration 3000000


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [4]:
for f in os.listdir(path_data):
    print(f)
    full_path = path_data+'/'+f
    try:
        f_text = open(full_path, "r").readlines()
        vectors=[]
        lines=[]
        for line in f_text:
            words_text=clean_text(line)
            emb_vector=np.zeros(400)
            for word in words_text:
                try:
                    emb_vector+=embdict[word]
                except:
                    pass
            lines.append(line)
            vectors.append(emb_vector)
        df=pd.DataFrame(vectors)
        df.insert(loc=0, column='texts', value=lines)
        df.to_csv(f+'.csv',sep=';')
    except:
        pass
    
print("Всего слов: "+str(len(words)))

nm_train.txt
np_val.txt
p_test.txt
i_test.txt
ni_train.txt
np_train.txt
nm_val.txt
ni_test.txt
i_val.txt
np_test.txt
nm_test.txt
none-class-1200.txt
p_val.txt
m_train.txt
i_train.txt
csv
m_val.txt
m_test.txt
p_train.txt
ni_val.txt
Всего слов: 10666


<h1>дальше код про сеть

In [5]:
import numpy as np
import tensorflow.keras as keras
import pandas as pd
import sklearn
from sklearn import preprocessing
from sklearn import svm
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Embedding, LSTM
from tensorflow.keras import callbacks
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import Dense, Embedding, LSTM, Flatten, GRU, SimpleRNN
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras import utils
import random
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
import copy

In [6]:
name_train1=r"none-class-1200.txt.csv"
name_train2=r"i_train.txt.csv"
name_test=r"i_test.txt.csv"
name_val=r"p_val.txt.csv"

In [7]:
with open(name_train1, encoding='utf-8') as f:
        df1 = pd.read_csv(f, sep=';', header=0, decimal = '.', index_col=0)
with open(name_train2, encoding='utf-8') as f:
        df2 = pd.read_csv(f, sep=';', header=0, decimal = '.', index_col=0)       

train_data1 = df1.values
train_data2 = df2.values

train_data1 = np.column_stack((np.zeros(len(train_data1)),train_data1))
train_data2 = np.column_stack((np.ones(len(train_data2)),train_data2))

train_data=np.vstack((train_data1,train_data2)) 
np.random.shuffle(train_data)

with open(name_test, encoding='utf-8') as f:
        df1 = pd.read_csv(f, sep=';', header=0, decimal = '.', index_col=0)
with open(name_val, encoding='utf-8') as f:
        df2 = pd.read_csv(f, sep=';', header=0, decimal = '.', index_col=0)
test_data = df1.values
test_data = np.column_stack((np.ones(len(test_data)),test_data))
val_data = df2.values
val_data = np.column_stack((np.ones(len(val_data)),val_data))

In [8]:
def remove_floats(texts, categories, vectors):
    _texts=[]
    _categories=[]
    _vectors=[]
    for i in range(len(texts)):
        if type(texts[i]) is str:
            _texts.append(clean_text(texts[i]))
            _categories.append(categories[i])
            _vectors.append(vectors[i])
    return _texts,_categories,_vectors

df = pd.DataFrame(train_data)
texts = df[1].tolist()
categories = df[0].tolist()
vectors = df.drop([0,1], axis=1).values
texts,categories,vectors=remove_floats(texts,categories,vectors)

df = pd.DataFrame(test_data)
texts_test = df[1].tolist()
categories_test = df[0].tolist()
vectors_test = df.drop([0,1], axis=1).values
texts_test,categories_test,vectors_test=remove_floats(texts_test,categories_test,vectors_test)

df = pd.DataFrame(val_data)
texts_val = df[1].tolist()
categories_val = df[0].tolist()
vectors_val = df.drop([0,1], axis=1).values
texts_val,categories_val,vectors_val=remove_floats(texts_val,categories_val,vectors_val)

In [12]:
num_classes = 2

descriptions = texts+texts_test
    
x_train = texts
y_train = categories
    
x_test = texts_test
y_test = categories_test

x_val = texts_val
y_val = categories_val


y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
y_val = keras.utils.to_categorical(y_val, num_classes)

max_words = 0
for desc in descriptions:
    try:
        words = len(desc)
        if words > max_words:
            max_words = words
    except:
        pass
print('Максимальное количество слов в самом длинном тексте: {} слов'.format(max_words))

maxSequenceLength = max_words

t = Tokenizer()
    
t.fit_on_texts(descriptions)
vocab_size = len(t.word_index) + 1
encoded_docs_train = t.texts_to_sequences(x_train)
encoded_docs_test = t.texts_to_sequences(x_test)
encoded_docs_val = t.texts_to_sequences(x_val)
padded_docs_train = sequence.pad_sequences(encoded_docs_train, maxlen=maxSequenceLength)
padded_docs_test = sequence.pad_sequences(encoded_docs_test, maxlen=maxSequenceLength)
padded_docs_val= sequence.pad_sequences(encoded_docs_val, maxlen=maxSequenceLength)

total_unique_words = len(t.word_counts)
print('Всего уникальных слов в словаре: {}'.format(total_unique_words))

Максимальное количество слов в самом длинном тексте: 94 слов
Всего уникальных слов в словаре: 5907


In [10]:
embedding_matrix = np.zeros((vocab_size, 400))
for word, i in t.word_index.items():
    try:
        embedding_vector = embdict[word]
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    except:
        print(word)

've
'll
're
gamerg
peshawarattack
ericgarn
lecoincanard
thanksobama
torturereport
sonyhack
blacklivesmatt
handsupdontshoot
'the
nfuller
eatenal
swormx
anticonversionlaw
finalsweek
cfbplayoff
mufflerman
gharwapsi
absolutetosh
newyearnewm
cameronmustgo
todayimlov
'republican
'effect
giveitup
thisismiser
nosocialconsci
reallynotr
nomealfitt
studentloan
intelsa
pbuffkin
shameony
putyourtoysaway
badboyfriend
twittertim
d|have
tortugaroug
stuism
notjustic
fantasticfriday
thatscool
whatmoredoesonene
copycatsay
michiganman
sosnoff
acatholicpray
cgshore
quiet|but
convos|and
weaknesses|todestroy
ifussss
'iron
rexemoji
manitibaprob
meaningful|cough
vlog|
keepyourpromis
truebro
killerband
ineednewboot
racewar
lovenotwar
shouldofstayedinb
toomuchlov
'd
onlykid
goodtv
freespeechmov
kcroyal
winterkil
whatdoiexpect
nmucomputersarethebest
goodlead
thirdyearlif
'legendari
soorganis
denialcsgo
nashuffl
qarara
justbrows
typical|
iaukea
needanewroutin
feelshit
michaelbeeren
ilookreallygood
wishwehada
milst

In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Flatten, GRU, SimpleRNN
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import Bidirectional

model = Sequential()
model.add(Embedding(vocab_size, 400, weights=[embedding_matrix], input_length=maxSequenceLength, trainable=False))
#model.add(e)
#e = Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=maxSequenceLength, trainable=False)
#model.add(e)
#model.add(Flatten())
#model.add(Dense(200, activation='sigmoid'))
#model.add(Dropout=0.5)
#model.add(Embedding(300, maxSequenceLength))
model.add(Bidirectional(LSTM(200, dropout=0.4, recurrent_dropout=0.2, return_sequences=True)))
model.add(Bidirectional(LSTM(200, dropout=0.4, recurrent_dropout=0.2, return_sequences=True)))
model.add(Bidirectional(LSTM(200, dropout=0.4, recurrent_dropout=0.2, return_sequences=True)))
model.add(Bidirectional(LSTM(200, dropout=0.4, recurrent_dropout=0.2)))
model.add(Dense(num_classes, activation='softmax'))
# compile the model
rmsprop = RMSprop(lr=0.001, rho=0.9, epsilon=1e-6)
#model.compile(optimizer = rmsprop, loss = 'mean_squared_error', metrics=['mean_squared_error', 'mae'])
#model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
# summarize the model
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 94, 400)           2363200   
_________________________________________________________________
bidirectional_4 (Bidirection (None, 94, 400)           961600    
_________________________________________________________________
bidirectional_5 (Bidirection (None, 94, 400)           961600    
_________________________________________________________________
bidirectional_6 (Bidirection (None, 94, 400)           961600    
_________________________________________________________________
bidirectional_7 (Bidirection (None, 400)               961600    
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 802       
Total params: 6,210,402
Trainable params: 3,847,202
Non-trainable params: 2,363,200
__________________________________________________________

In [18]:
history = model.fit(padded_docs_train, y_train, epochs = 10, verbose=2, validation_data=(padded_docs_test, y_test))
predict = np.argmax(model.predict(padded_docs_test), axis=1)
answer = np.argmax(y_test, axis=1)
print('Accuracy: %f' % (accuracy_score(predict, answer)*100))
print('F1-score: %f' % (f1_score(predict, answer, average="macro")*100))
print('Precision: %f' % (precision_score(predict, answer, average="macro")*100))
print('Recall: %f' % (recall_score(predict, answer, average="macro")*100)) 

Train on 3222 samples, validate on 100 samples
Epoch 1/10
 - 178s - loss: 0.4643 - acc: 0.7682 - val_loss: 0.1758 - val_acc: 0.9100
Epoch 2/10
 - 179s - loss: 0.4512 - acc: 0.7725 - val_loss: 0.4226 - val_acc: 0.7900
Epoch 3/10
 - 179s - loss: 0.4635 - acc: 0.7731 - val_loss: 0.3362 - val_acc: 0.9000
Epoch 4/10
 - 177s - loss: 0.4379 - acc: 0.7914 - val_loss: 0.5022 - val_acc: 0.6800
Epoch 5/10
 - 176s - loss: 0.5502 - acc: 0.7207 - val_loss: 0.5259 - val_acc: 0.7500
Epoch 6/10
 - 176s - loss: 0.4435 - acc: 0.7827 - val_loss: 0.2560 - val_acc: 0.9100
Epoch 7/10
 - 176s - loss: 0.4317 - acc: 0.7843 - val_loss: 0.4818 - val_acc: 0.7800
Epoch 8/10
 - 176s - loss: 0.4268 - acc: 0.7948 - val_loss: 0.3345 - val_acc: 0.8500
Epoch 9/10
 - 176s - loss: 0.4222 - acc: 0.8017 - val_loss: 0.5486 - val_acc: 0.7500
Epoch 10/10
 - 176s - loss: 0.4210 - acc: 0.8020 - val_loss: 0.2570 - val_acc: 0.9000
Accuracy: 90.000000
F1-score: 47.368421
Precision: 45.000000
Recall: 50.000000


/home/ors/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/ors/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


# 10 epochs:
- ## test:
Accuracy: 87.000000<br>
F1-score: 46.524064<br>
Precision: 43.500000<br>
Recall: 50.000000<br>

- ## val:
Accuracy: 35.000000<br>
F1-score: 25.925926<br>
Precision: 17.500000<br>
Recall: 50.000000<br>

# 20 epochs:
- ## test:
Accuracy: 90.000000<br>
F1-score: 47.368421<br>
Precision: 45.000000<br>
Recall: 50.000000<br>

- ## val:
Accuracy: 28.000000<br>
F1-score: 21.875000<br>
Precision: 14.000000<br>
Recall: 50.000000<br>

In [19]:
predict = np.argmax(model.predict(padded_docs_val), axis=1)
answer = np.argmax(y_val, axis=1)
print('Accuracy: %f' % (accuracy_score(predict, answer)*100))
print('F1-score: %f' % (f1_score(predict, answer, average="macro")*100))
print('Precision: %f' % (precision_score(predict, answer, average="macro")*100))
print('Recall: %f' % (recall_score(predict, answer, average="macro")*100))  

Accuracy: 28.000000
F1-score: 21.875000
Precision: 14.000000
Recall: 50.000000


/home/ors/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/ors/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
